In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
cd drive/MyDrive

/content/drive/MyDrive


In [3]:
!pip install transformers
!pip install sentencepiece
!pip install bertopic

In [4]:
!pip install 'git+https://github.com/SKTBrain/KoBERT.git#egg=kobert_tokenizer&subdirectory=kobert_hf'

  Cloning https://github.com/SKTBrain/KoBERT.git to /tmp/pip-install-v4pj4u24/kobert-tokenizer_20bfc3c2551f4c6b94a78b1c314f812b
  Running command git clone --filter=blob:none --quiet https://github.com/SKTBrain/KoBERT.git /tmp/pip-install-v4pj4u24/kobert-tokenizer_20bfc3c2551f4c6b94a78b1c314f812b
  Resolved https://github.com/SKTBrain/KoBERT.git to commit 47a69af87928fc24e20f571fe10c3cc9dd9af9a3
  Preparing metadata (setup.py) ... done


In [103]:
from tqdm import tqdm
from sklearn.feature_extraction.text import CountVectorizer
from bertopic import BERTopic
import os
import json

In [104]:
import re

cond = re.compile('[가-힣]+')

def get_data(gallery):
  result = []
  for name in os.listdir(path='/content/drive/MyDrive/jolnon/' + gallery):
    with open('/content/drive/MyDrive/jolnon/' + gallery + '/' + name, 'r', encoding='utf-8') as f:
      data = json.load(f)
      valid = []
      for d in data['content'].split('\n'):
        res = []
        for word in d.split(' '):
          if cond.match(word) != None:
            res.append(word)
        d = ' '.join(res)
        if d and not d.replace(' ', '').isdecimal():
          valid.append(d)
      result.append('\n'.join([data['title']] + valid))
  return result

In [105]:
import os
import json
import re
from zipfile import ZipFile

cond = re.compile('[가-힣]+')

def get_data_from_zip(zip_name):
    zip_file_path = '/content/drive/MyDrive/jolnon/' + zip_name + '.zip'
    extract_path = '/content/' + zip_name

    # zip 파일 압축 해제
    with ZipFile(zip_file_path, 'r') as zip_ref:
        zip_ref.extractall(extract_path)

    result = []
    for name in os.listdir(path=extract_path):
        file_path = os.path.join(extract_path, name)
        if os.path.isdir(file_path):
            for name in os.listdir(path=file_path):
                with open(file_path + '/' + name, 'r', encoding='utf-8') as f:
                    data = json.load(f)
                    valid = []
                    for d in data['content'].split('\n'):
                        res = []
                        for word in d.split(' '):
                            if cond.match(word) != None:
                                res.append(word)
                        d = ' '.join(res)
                        if d and not d.replace(' ', '').isdecimal():
                            valid.append(d)
                    result.append('\n'.join([data['title']] + valid))
        else:
            with open(extract_path + '/' + name, 'r', encoding='utf-8') as f:
                data = json.load(f)
                valid = []
                for d in data['content'].split('\n'):
                    res = []
                    for word in d.split(' '):
                        if cond.match(word) != None:
                            res.append(word)
                    d = ' '.join(res)
                    if d and not d.replace(' ', '').isdecimal():
                        valid.append(d)
                result.append('\n'.join([data['title']] + valid))
    return result


In [106]:
import random
random.seed(42)
galleries = ['giants_new2','cock_tail']#,'ktwiz','kartriderdrift','skwyverns_new1','ncdinos','samsunglions_new','doosanbears_new1','giants_new2','sh_new','lgtwins_new','tigers_new']
dataset = []
for e in galleries:
  dataset.append([get_data_from_zip(e)])
cocktail = []
for data in dataset:
  for sen in data:
    cocktail+=sen

In [107]:
def remove_words(sentence, word):
    return ' '.join([w for w in sentence.split(' ') if word not in w.lower()])
stopwords = ['ㅋㅋ', 'ex', '나는', 'on', '이미지', '순서', '오늘', '일단', 'and', '이야', '그리고', '내일', '그냥', '000', '조금', '살짝', 'ㅇㅇ', 'ㅈㄱㄴ', '있음', '이거', '내가', '칵하하하', '칵하핫', '근데', '지듣노', 'youtube ', '야스중', '우흥', '한다', 'ㅎㅎ', 'ㅠㅠ', '로오오오오오옹', '하고', '아침', '것도', '추천', '혹시', '새낀데', '같다']
for i in range(len(cocktail)):
  for j in range(len(stopwords)):
    cocktail[i] = remove_words(cocktail[i],stopwords[j])

In [108]:
import collections

def remove_top_n_words(data, n):
    words = [word for sentence in data for word in sentence.split(' ')]
    word_count = collections.Counter(words)
    top_n_words = [word for word, _ in word_count.most_common(n)]
    for i in range(len(data)):
        for word in top_n_words:
            if word=='':
                continue
            data[i] = remove_words(data[i], word)
    return data

def remove_empty_sentences(cocktail, real_labels):
  indices = [i for i, sentence in enumerate(cocktail) if sentence.strip() != '']
  cocktail = [cocktail[i] for i in indices]
  real_labels = [real_labels[i] for i in indices]
  return cocktail, real_labels


In [109]:
real_labels = []
for i in range(len(dataset)):
  real_labels += [i] * len(dataset[i][0])
cocktail = remove_top_n_words(cocktail, 20)
cocktail, real_labels = remove_empty_sentences(cocktail, real_labels)

In [110]:
from sklearn.metrics import confusion_matrix
import numpy as np
from scipy.optimize import linear_sum_assignment
from sklearn.metrics import confusion_matrix

def calc_purity(y_true, y_pred):
    cm = confusion_matrix(y_true, y_pred)
    _, col_ind = linear_sum_assignment(cm, maximize=True)
    reordered_cm = cm[:, col_ind]
    purity = np.sum(np.max(reordered_cm, axis=0)) / np.sum(reordered_cm)
    return purity, reordered_cm

In [115]:
bertopic_model = BERTopic(language='multilingual',
                          nr_topics='auto',
                          top_n_words=3,
                          calculate_probabilities=True)
topics, probs = bertopic_model.fit_transform(cocktail)

from sklearn.cluster import KMeans
n_clusters = len(dataset)
kmeans = KMeans(n_clusters=n_clusters, random_state=0)
kmeans.fit(probs)
labels = kmeans.labels_

from sklearn.metrics import confusion_matrix
print(len(real_labels))
print(confusion_matrix(real_labels, labels))

purity, reordered_cm = calc_purity(real_labels, labels)
print("Purity: ", purity)
print("Reordered Confusion Matrix: \n", reordered_cm)

4789
[[ 881 1182]
 [1232 1494]]
Purity:  0.569221131760284
Reordered Confusion Matrix: 
 [[1182  881]
 [1494 1232]]


In [120]:
# bertopic_model.visualize_topics()
# bertopic_model.visualize_distribution(probs[0])

In [116]:
from kobert_tokenizer import KoBERTTokenizer
tokenizer = KoBERTTokenizer.from_pretrained('skt/kobert-base-v1')
tokenizer.encode("한국어 모델을 공유합니다.")

!pip install 'git+https://github.com/SKTBrain/KoBERT.git#egg=kobert_tokenizer&subdirectory=kobert_hf'
import torch
from transformers import BertModel
model = BertModel.from_pretrained('skt/kobert-base-v1')

The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'XLNetTokenizer'. 
The class this function is called from is 'KoBERTTokenizer'.


  Cloning https://github.com/SKTBrain/KoBERT.git to /tmp/pip-install-l5edaxni/kobert-tokenizer_7917b72dcc274314b4f4e90ec86315fc
  Running command git clone --filter=blob:none --quiet https://github.com/SKTBrain/KoBERT.git /tmp/pip-install-l5edaxni/kobert-tokenizer_7917b72dcc274314b4f4e90ec86315fc
  Resolved https://github.com/SKTBrain/KoBERT.git to commit 47a69af87928fc24e20f571fe10c3cc9dd9af9a3
  Preparing metadata (setup.py) ... done


In [117]:
bertopic_model_embedding = BERTopic(embedding_model=model,
                                    nr_topics='auto',
                                    top_n_words=3,
                                    calculate_probabilities=True)
topics, probs = bertopic_model_embedding.fit_transform(cocktail)
n_clusters = len(dataset)
kmeans = KMeans(n_clusters=n_clusters, random_state=0)
kmeans.fit(probs)
labels = kmeans.labels_
print(len(real_labels))
print(confusion_matrix(real_labels, labels))

purity, reordered_cm = calc_purity(real_labels, labels)
print("Purity: ", purity)
print("Reordered Confusion Matrix: \n", reordered_cm)

4789
[[1935  128]
 [2590  136]]
Purity:  0.569221131760284
Reordered Confusion Matrix: 
 [[ 128 1935]
 [ 136 2590]]


In [118]:
class CustomTokenizer:
  def __init__(self, tokenizer):
    self.tokenizer = tokenizer
  def __call__(self, target):
    return self.tokenizer.tokenize(target)
from sklearn.feature_extraction.text import TfidfVectorizer

custom_tokenizer = CustomTokenizer(tokenizer)
# vectorizer = CountVectorizer(tokenizer=custom_tokenizer, max_features=3000)
vectorizer = TfidfVectorizer(tokenizer=custom_tokenizer, max_features=3000)

bertopic_model_tokenizer = BERTopic(vectorizer_model=vectorizer,
                                    nr_topics='auto',
                                    top_n_words=3,
                                    calculate_probabilities=True)
topics, probs = bertopic_model_tokenizer.fit_transform(cocktail)
n_clusters = len(dataset)
kmeans = KMeans(n_clusters=n_clusters, random_state=0)
kmeans.fit(probs)
labels = kmeans.labels_
print(len(real_labels))
print(confusion_matrix(real_labels, labels))

purity, reordered_cm = calc_purity(real_labels, labels)
print("Purity: ", purity)
print("Reordered Confusion Matrix: \n", reordered_cm)

4789
[[1264  799]
 [1751  975]]
Purity:  0.569221131760284
Reordered Confusion Matrix: 
 [[ 799 1264]
 [ 975 1751]]


In [119]:
bertopic_model_embed_token = BERTopic(embedding_model=model,
                                      vectorizer_model=vectorizer,
                                      nr_topics='auto',
                                      top_n_words=3,
                                      calculate_probabilities=True)
topics, probs = bertopic_model_embed_token.fit_transform(cocktail)
n_clusters = len(dataset)
kmeans = KMeans(n_clusters=n_clusters, random_state=0)
kmeans.fit(probs)
labels = kmeans.labels_
print(len(real_labels))
print(confusion_matrix(real_labels, labels))

purity, reordered_cm = calc_purity(real_labels, labels)
print("Purity: ", purity)
print("Reordered Confusion Matrix: \n", reordered_cm)

4789
[[1946  117]
 [2595  131]]
Purity:  0.569221131760284
Reordered Confusion Matrix: 
 [[ 117 1946]
 [ 131 2595]]
